# Bibliotecas

In [1]:
import numpy as np

from ml_core.cross_validation import Kfold
from ml_core.classification.non_parametric.base_class import NonParametricAlg
from ml_core.metrics.distance_metrics import euclidian, Mahalanobis
from ml_core.classification.non_parametric.knn import Knn
import pandas as pd
from ml_core.data_process import *
from sklearn.tree import DecisionTreeClassifier
import pandas_profiling as pp
from ml_core.normalization import z_score_normalization
from sklearn.model_selection import train_test_split
from ml_core.metrics.error_metrics import acuracy, f1_score, precision

# Dados

In [2]:
df = pd.read_csv('data/kc2.csv', header=None)

In [3]:
df

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,1.1,1.4,1.4,1.4,1.3,1.30,1.30,1.30,1.30,1.30,...,2.0,2.0,2.0,2.0,1.2,1.2,1.2,1.2,1.4,0.0
1,1.0,1.0,1.0,1.0,1.0,1.00,1.00,1.00,1.00,1.00,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,415.0,59.0,50.0,51.0,1159.0,8411.31,0.01,103.53,81.24,870848.58,...,359.0,35.0,9.0,10.0,47.0,106.0,692.0,467.0,106.0,1.0
3,230.0,33.0,10.0,16.0,575.0,3732.82,0.03,39.82,93.74,148644.06,...,174.0,15.0,34.0,5.0,23.0,67.0,343.0,232.0,65.0,1.0
4,175.0,26.0,12.0,13.0,500.0,3123.96,0.03,29.48,105.96,92103.07,...,142.0,7.0,19.0,4.0,18.0,58.0,310.0,190.0,51.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,4.0,1.0,1.0,1.0,5.0,11.61,0.50,2.00,5.80,23.22,...,2.0,0.0,0.0,0.0,4.0,1.0,4.0,1.0,1.0,1.0
518,4.0,1.0,1.0,1.0,4.0,8.00,0.67,1.50,5.33,12.00,...,2.0,0.0,0.0,0.0,3.0,1.0,3.0,1.0,1.0,1.0
519,4.0,1.0,1.0,1.0,4.0,8.00,0.67,1.50,5.33,12.00,...,2.0,0.0,0.0,0.0,3.0,1.0,3.0,1.0,1.0,1.0
520,4.0,1.0,1.0,1.0,5.0,11.61,0.67,1.50,7.74,17.41,...,2.0,0.0,0.0,0.0,3.0,2.0,3.0,2.0,1.0,1.0


# Profile

In [4]:
# profile = pp.ProfileReport(df, title="Demo de EDA Automatico", explorative=True)

In [5]:
# profile.to_file("meu_report.html")

# Remoção de Duplicatas

In [6]:
df_unique = df.drop_duplicates()

In [7]:
df_unique

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,1.1,1.4,1.4,1.4,1.3,1.30,1.30,1.30,1.30,1.30,...,2.0,2.0,2.0,2.0,1.2,1.2,1.2,1.2,1.4,0.0
1,1.0,1.0,1.0,1.0,1.0,1.00,1.00,1.00,1.00,1.00,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,415.0,59.0,50.0,51.0,1159.0,8411.31,0.01,103.53,81.24,870848.58,...,359.0,35.0,9.0,10.0,47.0,106.0,692.0,467.0,106.0,1.0
3,230.0,33.0,10.0,16.0,575.0,3732.82,0.03,39.82,93.74,148644.06,...,174.0,15.0,34.0,5.0,23.0,67.0,343.0,232.0,65.0,1.0
4,175.0,26.0,12.0,13.0,500.0,3123.96,0.03,29.48,105.96,92103.07,...,142.0,7.0,19.0,4.0,18.0,58.0,310.0,190.0,51.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
516,6.0,2.0,1.0,1.0,15.0,53.77,0.40,2.50,21.51,134.44,...,2.0,0.0,2.0,0.0,5.0,7.0,8.0,7.0,3.0,1.0
517,4.0,1.0,1.0,1.0,5.0,11.61,0.50,2.00,5.80,23.22,...,2.0,0.0,0.0,0.0,4.0,1.0,4.0,1.0,1.0,1.0
518,4.0,1.0,1.0,1.0,4.0,8.00,0.67,1.50,5.33,12.00,...,2.0,0.0,0.0,0.0,3.0,1.0,3.0,1.0,1.0,1.0
520,4.0,1.0,1.0,1.0,5.0,11.61,0.67,1.50,7.74,17.41,...,2.0,0.0,0.0,0.0,3.0,2.0,3.0,2.0,1.0,1.0


# Separando Arrays

In [8]:
arr = df_unique.to_numpy()

In [9]:
X = arr[:, :-1]
y = arr[:, -1].reshape([-1,1])

In [10]:
X_norm = np.empty_like(X)

for idx, col in enumerate(X.T):

    norm = z_score_normalization(col)

    X_norm[:, idx] = norm

In [11]:
X_trn, X_tst, y_trn, y_tst = train_test_split(X_norm, y, test_size=0.2, random_state=42)


# Modelos

# KNN - Euclidiano

In [12]:
knn_euclidian = Knn(
    X_trn,
    y_trn,
    4,
    euclidian
)

In [13]:
preds_euclidian = knn_euclidian.predict(X_tst)

In [14]:
acuracy(y_tst, preds_euclidian), f1_score(y_tst, preds_euclidian), precision(y_tst, preds_euclidian)

(0.84, 0.345679012345679, 0.2222222222222222)

# KNN - Mahalanobis

In [15]:
cov_x = np.cov(X_trn.T)

In [16]:
mahalanobis = Mahalanobis(cov_x)

In [17]:
knn_mahalanobis = Knn(
    X_trn,
    y_trn,
    4,
    mahalanobis
)

In [18]:
preds_mahalanobis = knn_mahalanobis.predict(X_tst)

In [19]:
acuracy(y_tst, preds_mahalanobis), f1_score(y_tst, preds_mahalanobis), precision(y_tst, preds_mahalanobis)

(0.8, 0.15384615384615383, 0.1)

# Árvores de Decisão

In [20]:
tree_clf = DecisionTreeClassifier(max_depth=2)

In [21]:
tree_model = tree_clf.fit(X_trn, y_trn)

In [22]:
preds_tree = tree_model.predict(X_tst).reshape([-1,1])

In [23]:
acuracy(y_tst, preds_tree), f1_score(y_tst, preds_tree), precision(y_tst, preds_tree)

(0.84, 0.3950617283950617, 0.25396825396825395)

# K Fold

In [36]:
idx = np.arange(X_norm.shape[0])

np.random.seed(42)
np.random.shuffle(idx)

X_shuffle = X_norm[idx]
y_shuffle = y[idx]

In [37]:
knn_metrics = {
    'Acurácia': acuracy,
    'F1-Score': f1_score,
    'Precisão': precision
}

In [38]:
dt_metrics = {
    'Acurácia': lambda x, y: acuracy(x, y.reshape([-1,1])),
    'F1-Score': lambda x, y: f1_score(x, y.reshape([-1,1])),
    'Precisão': lambda x, y: precision(x, y.reshape([-1,1]))
}

# KNN - Euclidiano

In [39]:
k_fold_knn = Kfold(k=10, metrics=knn_metrics, verbose=True)

In [40]:
alg_knn = NonParametricAlg(
    Knn,
    k=5,
    dist_function=euclidian
)

In [41]:
_ = k_fold_knn(alg_knn, X_shuffle, y_shuffle)

--------------REPORTANDO OS RESULTADOS OBTIDOS--------------
--------------ACURÁCIA--------------
Média: 0.7438833570412519
Desvio Padrão: 0.07485074959726377
--------------F1-SCORE--------------
Média: 0.26769303424252333
Desvio Padrão: 0.08063508867176257
--------------PRECISÃO--------------
Média: 0.1834501963800787
Desvio Padrão: 0.0506778081119021


# KNN - Mahalanobis

In [42]:
def ajust_mahalanobis(ml_alg, data):
    cov_x = np.cov(data['x_trn'].T)
    mahalanobis = Mahalanobis(cov_x)

    model = ml_alg.fit(
			data['x_trn'],
			data['y_trn']
	)

    model.dist_function = mahalanobis

    return model

In [43]:
alg_knn_mahalanobis = NonParametricAlg(
    Knn,
    k=5,
    dist_function=mahalanobis
)

In [44]:
hist = k_fold_knn(alg_knn_mahalanobis, X_shuffle, y_shuffle, apply_alg_function=ajust_mahalanobis)

--------------REPORTANDO OS RESULTADOS OBTIDOS--------------
--------------ACURÁCIA--------------
Média: 0.784068278805121
Desvio Padrão: 0.038306861103030096
--------------F1-SCORE--------------
Média: 0.2093377096714709
Desvio Padrão: 0.05901009231417996
--------------PRECISÃO--------------
Média: 0.14290585218143617
Desvio Padrão: 0.04313302198637145


# Arvore de Decisão

In [45]:
k_fold_dt = Kfold(k=10, metrics=dt_metrics, verbose=True)

In [46]:
tree_clf = DecisionTreeClassifier(max_depth=5)

In [47]:
_ = k_fold_dt(tree_clf, X_shuffle, y_shuffle)

--------------REPORTANDO OS RESULTADOS OBTIDOS--------------
--------------ACURÁCIA--------------
Média: 0.7363442389758179
Desvio Padrão: 0.046509769985574136
--------------F1-SCORE--------------
Média: 0.2538269935343774
Desvio Padrão: 0.07174719535271042
--------------PRECISÃO--------------
Média: 0.17603506270024624
Desvio Padrão: 0.054248782950239986
